# Cryostation initialization

A cryostation is a device for storing viruses in the real world.  Cryostations keep virus samples (different locations or generations) for later use in vaccine or gene sequencing research.

The `covidvu.cryostation` module will be used for storing all the processed data sources and manipulation results, for quick bundling and access.

## TinyDB

- [API reference](https://tinydb.readthedocs.io/en/latest/api.html)
- [Documentation](https://tinydb.readthedocs.io/en/latest/index.html)


---
## General purpose functions

In [ ]:
import json
import os

In [ ]:
from covidvu.cryostation import Cryostation

In [ ]:
from covidvu.pipeline.vujson import SITE_DATA

In [ ]:
def loadDatasetFrom(source = 'confirmed', suffix = ''):
    fileName = os.path.join(SITE_DATA, source+'%s.json' % suffix)
    
    return json.loads(open(fileName, 'r').read())

In [ ]:
def updateCasesIn(cryostation, casesType = 'confirmed'):
    sourceDataset = loadDatasetFrom(casesType)
    
    for key in sourceDataset.keys():
        item = cryostation.get(key, dict())
        if not key in item:
            item = dict()
            item['key'] = key
            
        item[casesType] = sourceDataset[key]
        
        cryostation[key] = item

In [ ]:
storage = Cryostation('database/virustrack.db')

---
## First time use

The routines in this notebook are used as a one-time only database initialization, based on the existing data bundles.

## Global cases

In [ ]:
updateCasesIn(storage, 'confirmed')

In [ ]:
updateCasesIn(storage, 'deaths')

In [ ]:
storage.close()

### Capitals, ISO codes, languages

In [ ]:
from covidvu.virustrack.countryinfo import COUNTRIES_INFO

import time

In [ ]:
storage = Cryostation('database/virustrack.db')

In [ ]:
for country in COUNTRIES_INFO.keys():
    countryInfo = COUNTRIES_INFO[country]
    if country not in storage:
        print('MISSING: %s' % country)
        missingCountry = { 'key': country, }
        storage[country] = missingCountry

    element = storage[country]
    element['info'] = countryInfo
    print('%s: %s' % (country, element['info']['capital']))
    storage[country] = element

In [ ]:
storage.close()

---
## US states

In [ ]:
def update(element, casesType, suffix = '-US'):
    dataset = loadDatasetFrom(casesType, suffix)   
    for state in dataset.keys():
        if state not in element:
            element[state] = dict()
        
        element[state][casesType] = dataset[state]
    
    return element

In [ ]:
storage = Cryostation('database/virustrack.db')

In [ ]:
element = storage['US']

In [ ]:
element = update(element, 'confirmed')

In [ ]:
element = update(element, 'deaths')

In [ ]:
storage['US'] = element

In [ ]:
storage.close()

### US Regions

In [ ]:
storage = Cryostation('database/virustrack.db')

In [ ]:
element = storage['US']

In [ ]:
element = update(element, 'confirmed', '-US-Regions')

In [ ]:
element = update(element, 'deaths', '-US-Regions')

In [ ]:
storage['US'] = element

In [ ]:
storage.close()

Each US state belongs to a region -- let's mark them here so the regions exist in the data, not in the code.

In [ ]:
from covidvu.virustrack.countryinfo import US_REGIONS

In [ ]:
storage = Cryostation('database/virustrack.db')

In [ ]:
element = storage['US']

In [ ]:
for state in US_REGIONS.keys():
    if state in element:
        element[state]['region'] = US_REGIONS[state]

In [ ]:
storage['US'] = element

In [ ]:
storage.close()

In [ ]:
storage = Cryostation('database/virustrack.db')
element = storage['US']


---
&#169; The VirusTrack/COVIDvu Open Source Development Team